# **Koopman Lifting Example**

In [3]:
from kq_lmpc_quadrotor import KoopmanLift
import numpy as np

Enter parameters of the quadrotor in the following format

In [5]:
Ixx = 0.002354405654  # kg-m^2
Iyy = 0.002629495468
Izz = 0.003186782158
x_rot = 0.175 / 2
y_rot = 0.131 / 2

params = {
                'mass': 0.904,  # kg
                "J": np.diag([Ixx, Iyy, Izz]),
                "d": np.sqrt(x_rot**2 + y_rot**2),
                "k_eta": 0.0000007949116377,
                "c_tau": 1.969658853113417e-8,
                "wr_max": 2500,
                "g": 9.81,  # gravity
                "e3": np.array([0, 0, 1]).reshape(-1,1)
            }

Initialize Koopmanlift

In [7]:
M, N = 3, 2 ## refere to the paper on what are M,N
koop = KoopmanLift(params, M, N)

Gnereate a random 18D state vector

In [8]:
from kq_lmpc_quadrotor import rand_state
x = rand_state()
print(x)

[[ 0.52399579]
 [ 0.17084951]
 [ 0.78652237]
 [-0.38085563]
 [ 0.48635275]
 [ 0.32625563]
 [ 0.98886297]
 [-0.12158454]
 [ 0.08583255]
 [ 0.13234437]
 [ 0.9821826 ]
 [-0.13342528]
 [-0.06808078]
 [ 0.14329878]
 [ 0.98733503]
 [ 0.00680548]
 [-0.0092032 ]
 [-0.00923828]]


Koopman lifting of x

In [10]:
X = koop.fcn_gen_koopman_states_se3(x)
print(X.shape)

## Let's recover x from X

x_reco = koop.fcn_se3_states_to_actual(X)
print(x_reco)

(45, 1)
[[ 0.52399579]
 [ 0.17084951]
 [ 0.78652237]
 [-0.38085563]
 [ 0.48635275]
 [ 0.32625563]
 [ 0.98886297]
 [-0.12158454]
 [ 0.08583255]
 [ 0.13234437]
 [ 0.9821826 ]
 [-0.13342528]
 [-0.06808078]
 [ 0.14329878]
 [ 0.98733503]
 [ 0.00680548]
 [-0.0092032 ]
 [-0.00923828]]


You can also fetch the paramters used in the paper

In [11]:
from kq_lmpc_quadrotor import UtilsFunctions
utilsf = UtilsFunctions()
params,_,_ = utilsf.get_quad_params()
print(params)

No input parameters provided ::: using default values..
	 M = 3, N = 3
{'mass': 0.904, 'J': array([[0.00235441, 0.        , 0.        ],
       [0.        , 0.0026295 , 0.        ],
       [0.        , 0.        , 0.00318678]]), 'd': np.float64(0.10930004574564459), 'k_eta': 7.949116377e-07, 'c_tau': 1.969658853113417e-08, 'wr_max': 2500, 'g': 9.81, 'e3': array([[0],
       [0],
       [1]])}


## LTI Formulation

In [12]:
# Get A , and B (Bbar) LTI matrices
A = koop.fcn_A_lifted()
B = koop.fcn_Bbar()

print(A.shape)
print(B.shape)

(45, 45)
(45, 28)


In [15]:
# Lifted control U
u = np.array([10, 0.1,0.1,0.1]) # consider a 4D control
U = koop.fcn_u2U(x,u)
print(U.shape)

(28, 1)


In [16]:
# Recover u from U
u_reco = koop.fcn_U2u(x,U)
print(u_reco)

[[10. ]
 [ 0.1]
 [ 0.1]
 [ 0.1]]


## Quasil-linear (LPV) formulation

In [17]:
# A matrix is the same
# Construct the state dependent B (calB) matrix
calB = koop.fcn_calB(x)
print(calB.shape)

(45, 4)


In [18]:
# construct the modified control u_tilde
u_tilde = koop.fcn_u2u_tilde(x,u)

In [19]:
# recover u from u_tilde
u_reco2 = koop.fcn_u_tilde2u(x,u_tilde)
print(u_reco2)

[[10. ]
 [ 0.1]
 [ 0.1]
 [ 0.1]]


## Symbolic Koopman Lifting (CasADi)

In [23]:
import casadi as ca

# consider an 18D state
x = ca.SX.sym('x',18,1)
print(x)

[x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12, x_13, x_14, x_15, x_16, x_17]


In [24]:
# lifted Symbolic state
X = koop.casadi_gen_koopman_states_se3(x)
print(X.shape)

(45, 1)


In [26]:
# Recovering original x
x_reco = koop.casadi_se3_to_actual(X)
print(x_reco.shape)

(18, 1)


In [28]:
# Symbolic control input transformations
u = ca.SX.sym('u',4,1)

# lifted control input
U = koop.casadi_u2U(x,u)
print(U.shape)

# recovering u from U
u_reco = koop.casadi_U2u(x,U)
print(u_reco.shape)

# constructing modified u_tilde
u_tilde = koop.casadi_u2u_tilde(x,u)
print(u_tilde.shape)

# Recovering u from u_tilde
u_reco2 = koop.casadi_u_tilde2u(x,u_tilde)
print(u_reco.shape)

(28, 1)
(4, 1)
(4, 1)
(4, 1)


In [29]:
# Symbolic LPV B matrix (calB)

calB_sym = koop.casadi_calB(x)
print(calB_sym.shape)

(45, 4)
